In [41]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import f1_score

In [42]:
train = pd.read_csv("../Data/TaskA_train_robert.csv", index_col=False)
test = pd.read_csv("../Data/TaskA_test_robert.csv", index_col=False)

In [43]:
X_train = train.loc[:, ["topic", "Premise", "Conclusion", "RoBERT_premise", "RoBERT_conclusion", "RoBERT_cosine_sim"]]
y_train = train.loc[:, ["Validity", "Novelty"]]
X_test = test.loc[:, ["topic", "Premise", "Conclusion", "RoBERT_premise", "RoBERT_conclusion", "RoBERT_cosine_sim"]]
y_test = test.loc[:, ["Validity", "Novelty"]]

In [72]:
def str_to_list(text):
    '''
    Return tensor string into list
    '''
    # clean string
    clean_str = text.replace('tensor(', '').replace(')', '').strip()
    # convert to list
    tensor = eval(clean_str, {"torch": torch, "__builtins__": {}})
    return tensor

def process_covariate_data(df):
    '''
    Expanding all tensors in a single cell
    '''
    # convert str to tensor (list)
    RoBERT_premise = df.RoBERT_premise.apply(lambda x: str_to_list(x))
    RoBERT_conclusion = df.RoBERT_conclusion.apply(lambda x: str_to_list(x))
    RoBERT_cosine_sim = df.RoBERT_cosine_sim.apply(lambda x: float(x[2:-3]))

    # expand the list into individual entries
    df_expand1 = RoBERT_premise.apply(pd.Series)
    df_expand2 = RoBERT_conclusion.apply(pd.Series)

    # assign a meaningful name
    df_expand1.columns = ['pre_emb{}'.format(i+1) for i in range(df_expand1.shape[1])]
    df_expand2.columns = ['con_emb{}'.format(i+1) for i in range(df_expand2.shape[1])]

    # put everything together
    df_final = pd.concat([df.drop(['RoBERT_premise', "RoBERT_conclusion", "RoBERT_cosine_sim"], axis=1), df_expand1, df_expand2, RoBERT_cosine_sim], axis=1)
    return df_final

def preprocess_input(x, y):
    '''
    return DataLoader for later input into the model
    '''
    # pd.dataframe to array
    x = np.array(x, dtype=np.float64)
    y = np.array(y, dtype=np.float64)
    # transform y for nn model
    # assume a value of 0 is not valid/novel
    y[y == 0] = -1
    # Transform the data
    transformation_dict = {
    (1, 1): [1, 0, 0, 0],
    (1, -1): [0, 1, 0, 0],
    (-1, 1): [0, 0, 1, 0],
    (-1, -1): [0, 0, 0, 1],
    }
    y = np.array([transformation_dict[tuple(row)] for row in y])
    # array to tensor
    x_torch = torch.tensor(x)
    y_torch = torch.tensor(y)
    data = TensorDataset(x_torch, y_torch)

    batch_size = 10
    loader = DataLoader(data, batch_size=batch_size, shuffle=True)
    return loader

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.input_dim = 1537
        self.hidden_dim = 200
        self.output_dim = 4
        self.fc1 = nn.Linear(self.input_dim, self.hidden_dim)  # Assuming n input features
        self.fc2 = nn.Linear(self.hidden_dim, 4)  # Outputs 4 values, one for each output column
    def forward(self, x):
        x = x.float()
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [73]:
X_train1 = process_covariate_data(X_train)
X_test1 = process_covariate_data(X_test)

train_loader = preprocess_input(X_train1.iloc[:, 3:], y_train)
test_loader = preprocess_input(X_test1.iloc[:, 3:], y_test)

In [74]:
model = SimpleNN()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [75]:
epochs = 51
for epoch in range(epochs):
    # for each batch
    for inputs, targets in train_loader:
        inputs = inputs.float()
        targets = targets.float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

    # Optionally print the loss every few epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.14881061017513275
Epoch 11, Loss: 0.09726180136203766
Epoch 21, Loss: 0.08729208260774612
Epoch 31, Loss: 0.09401580691337585
Epoch 41, Loss: 0.018242282792925835
Epoch 51, Loss: 0.05889059230685234


In [76]:
model.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    test_loss = 0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        print(outputs.data)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        test_labels.extend(targets.tolist())
        test_preds.extend(predicted.tolist())
f1 = f1_score(test_labels, test_preds, average='macro')

tensor([[-1.6032e-01,  3.5528e-02,  4.9553e-03,  1.1385e+00],
        [-3.0650e-01,  4.6352e-02,  1.3555e-03,  1.2824e+00],
        [ 7.0479e-03,  6.0095e-02,  1.6899e-02,  9.3092e-01],
        [ 4.8105e-02,  9.1574e-03,  8.9871e-03,  9.4499e-01],
        [ 2.6001e-01, -6.9122e-02,  2.9745e-03,  8.0722e-01],
        [-9.1543e-03,  3.0433e-01,  2.0619e-02,  7.0325e-01],
        [-1.0651e-01, -1.8124e-02, -1.4502e-02,  1.1512e+00],
        [-9.3974e-02,  8.5984e-01, -3.1269e-02,  2.8340e-01],
        [-2.1714e-01,  1.7502e-02, -3.9528e-04,  1.2196e+00],
        [ 1.4276e-01,  1.2191e-01,  3.4166e-02,  7.1512e-01]])
tensor([3, 3, 3, 3, 3, 3, 3, 1, 3, 3])
tensor([[ 2.6566e-01,  1.4259e-01,  4.3142e-02,  5.5992e-01],
        [ 6.9503e-02, -6.2508e-02, -3.6470e-03,  1.0037e+00],
        [-4.4024e-02,  2.4868e-02,  8.1451e-03,  1.0258e+00],
        [-3.8992e-01,  3.2321e-02, -4.7370e-03,  1.3878e+00],
        [ 6.4483e-01,  2.0636e-01,  7.0834e-02,  8.1122e-02],
        [ 2.6513e-01, -3.5494e

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [ ]:
test_preds

[0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
